### AlexNet

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
mnist = input_data.read_data_sets('/tmp/data/',one_hot=True)

W0921 14:50:07.123512  6244 deprecation.py:323] From <ipython-input-2-eb4adea6bc33>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0921 14:50:07.127532  6244 deprecation.py:323] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0921 14:50:07.128497  6244 deprecation.py:323] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Pl

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz


W0921 14:50:07.338956  6244 deprecation.py:323] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.001
training_iters = 300000
batch_size = 64
display_step = 100
n_input = 784
n_classes = 10
dropout = 0.8

In [6]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [8]:
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1],
                                                  padding='SAME'),b), name=name)

def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME', name=name)

# local_response_normalization 약자
def norm(name, l_input, lsize=4):
    # 사이즈를 고려한 방향
    # alpha: scale factor, beta: 0.5 default 지수형 값을 준다
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

def alex_net(_X, _weights, _biases, _dropout):
    
    # 흑백이미지, 마지막이 1, 몇개가 될지는 뒤의 차수로 결정 -1
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])
    
    # 가중치는 filter 개수만큼 생성  = 나가는 차수 동일
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    
    # 2x2로 풀링 1/4크기
    pool1 = max_pool('pool1', conv1, k=2)

    # 정규화 (벡터의 크기로 데이터를 나눠줌)
    # vector = 1, 방향값만을 나타내는 내적의 용이 -> 각도 측정
    norm1 = norm('norm1', pool1, lsize=4)
    
    # dropout 계산 회선수를 줄임(과적합 방지)
    norm1 = tf.nn.dropout(norm1, _dropout)

    # Conv 2 Layer
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    pool2 = max_pool('pool2', conv2, k=2)
    norm2 = norm('norm2', pool2, lsize=4)
    norm2 = tf.nn.dropout(norm2, _dropout)

    # Conv 3 Layer
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    pool3 = max_pool('pool3', conv3, k=2)
    norm3 = norm('norm3', pool3, lsize=4)
    norm3 = tf.nn.dropout(norm3, _dropout)
    
    # flattening함
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]])
    
    # matmul 시 항상 bias 항상 고려(절편, 0으로 가는 것을 방지)
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1')
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2')
    
    # 3개의 FFNN 레이어 사용하여 분류
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    
    return out

In [9]:
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64])),  
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128])), 
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wd1': tf.Variable(tf.random_normal([4*4*256, 1024])),  #배치사이즈*4,096 내적 4,096 -> 1,024 로 축약 = batch_size*1,024
    'wd2': tf.Variable(tf.random_normal([1024, 1024])), # 배치사이즈x1024 내적 1024x1024
    'out': tf.Variable(tf.random_normal([1024, 10])) # 배치사이즈x1024 내적 1024x10 = 배치사이즈x10
}
biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([128])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# 데이터, 가중치, 바이어스, 드랍아웃
pred = alex_net(x, weights, biases, keep_prob) #예측

# 예측치  실제값 차 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
     logits=pred, labels=y)) # 차이 측정

# optimizer = learning_rate  매개변수
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) #훈련

# 맞으면 1, 틀리면 0
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) #정확도
init = tf.initialize_all_variables()

W0921 15:52:05.215910  6244 deprecation.py:506] From <ipython-input-8-eb31d673b6a2>:31: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0921 15:52:05.515343  6244 deprecation.py:323] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\python\util\tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [10]:
with tf.Session() as sess:
    # 가중치 있으면 무조건 하는거래
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " \
                  + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                                             y: mnist.test.labels[:256],
                                                             keep_prob: 1.})) 

Iter 6400, Minibatch Loss= 19961.816406, Training Accuracy= 0.75000
Iter 12800, Minibatch Loss= 21957.671875, Training Accuracy= 0.56250
Iter 19200, Minibatch Loss= 24611.845703, Training Accuracy= 0.75000
Iter 25600, Minibatch Loss= 18732.570312, Training Accuracy= 0.78125
Iter 32000, Minibatch Loss= 10321.678711, Training Accuracy= 0.75000
Iter 38400, Minibatch Loss= 8849.416992, Training Accuracy= 0.73438
Iter 44800, Minibatch Loss= 6058.571777, Training Accuracy= 0.85938
Iter 51200, Minibatch Loss= 6399.568359, Training Accuracy= 0.85938
Iter 57600, Minibatch Loss= 9829.583984, Training Accuracy= 0.84375
Iter 64000, Minibatch Loss= 7138.193359, Training Accuracy= 0.79688
Iter 70400, Minibatch Loss= 5724.901367, Training Accuracy= 0.82812
Iter 76800, Minibatch Loss= 2802.986572, Training Accuracy= 0.89062
Iter 83200, Minibatch Loss= 674.084595, Training Accuracy= 0.96875
Iter 89600, Minibatch Loss= 4900.488281, Training Accuracy= 0.93750
Iter 96000, Minibatch Loss= 4505.767578, Trai

KeyboardInterrupt: 